# ENGR 326 Lab 6: Systems of ODEs for wastewater treatment

In this lab we will use RK4 to solve a system of ODEs.

Figure formatting: 
* Please make sure all of your figures include axis labels that include units. 
* If you are plotting multiple curves on one graph, make sure to include a legend.
* Make sure to use LaTeX formatting in axis labels and legends.
* Do NOT put titles on your figures.


## 1. Problem statement
As part of a new discharge permit, a wastewater treatment facility is required to reduce the phosphorus concentration in the water it releases to a small stream channel. This discharged water from the treatment facility into the stream is called the "effluent". 
 
The channel flow is assumed to be dominated by the facility discharge (i.e., upstream flow is negligible), so the initial in-stream phosphorus concentration at the discharge point equals the effluent concentration. The current effluent phosphorus concentration is 0.90 $\frac{mg}{L}$.

The new effluent limit will be determined based on the requirement that the phosphorus concentration in the stream must not exceed 0.05 $\frac{mg}{L}$ at a location 44.5 $km$ downstream of the discharge point. This location was selected because it is where the stream connects to a larger river. This downstream criterion was established to protect designated beneficial uses, including recreation and drinking water supply.

## 2. Objectives
The objectives of this lab are as follows:

1. *Original alongstream phosphorus concentration:* Plot and describe how the concentration of phosphorus changes from the discharge point to the downstream regulation point using the current concentration in the treatment plant effluent.

2. *Original downstream phosphorus concentration:* Determine the concentration of phosphorus at the downstream regulation point using the current concentration in the treatment plant effluent.

3. *Max allowable phosphorus effluent concentration:* Determine the new maximum concentration of phosphorus allowed in the treatment plant
effluent that meets the downstream limit.

4. *Max regulated alongstream phosphorus concentration*: Plot and describe how the concentration of phosphorus changes from the discharge point to the downstream
regulation point using the new effluent concentration that meets the downstream limit.

We will also perform code validation and sensitivity analyses to ensure our methods are reliable.

## 3. Math Model

The engineer in charge of the treatment plant upgrade has determined that phosphorus in the stream is removed by two mechanisms: algal uptake and settling of suspended particles. They have also determined that phosphorus is the limiting nutrient for algal growth in the stream, and that the algae growth rate can be estimated using the Michaelis–Menten saturation kinetics (Nyholm, 1977). They have identified the following equations (Loucks and Van Beek, 2017) as appropriate for modeling the phosphorus concentration and chlorophyll-A (a measure of algae concentration) in the stream:

$$\frac{dP}{dt} = -K_{P1}P - K_{P2} \mu A \tag{1}$$

$$\frac{dA}{dt} = -K_{A1}A + \mu A \tag{2}$$

where,

$$ \mu = \mu_{max} \frac{P}{K_{P3}+P} \tag{3}$$

and where,

$P(t)$ is the phosphorus concentration in $\frac{mg}{L}$,

$A(t)$ is the chlorophyll concentration in $\frac{mg}{L}$,

$t$ is the time in $days$,

$K_{P1}$ is the first order removal rate of phosphorus by particle settling with units of $1/day$,

$K_{P2}$ is the yield coefficient that represents the amount of phosphorus consumed per unit mass of algae produced with units of $\frac{mg \ phosphorus}{mg \ chlorophyll-A}$,

$K_{A1}$ is the algal death rate with units of $1/day$,

$\mu(t)$ is the algal growth rate with units of $1/day$,

$\mu_{max}$ is the maximum algal growth rate with units of $1/day$,

and $K_{P3}$ is the half saturation concentration for phosphorus in $\frac{mg}{L}$.

Note that these equations are in a reference frame following a fluid parcel. Therefore in order to calculate concentrations at some location downstream, we must convert from time to distance using the stream velocity, $ x = v t$.

## 4. Numerical Method

The ODE system above is not solvable by hand, so we will need to use a numerical method to solve it.

### Question 4.1

*Double click on this box and write a description of the RK4 numerical method (which you will use to solve these ODEs).* 

*Make sure to include equations in your description.* 

*Also make sure to cite the textbook (Chapra & Canale, 2011). The reference for your textbook is provided at the bottom of this lab. Make sure to [paraphrase](https://youtu.be/-SqpAsNBE5A?si=3JgMqzJ_QW6RUyth) the textbook in a way that is not [plagiarism](https://youtu.be/Uk1pq8sb-eo?si=V5scUQ_oFOVCEcgh).*

## 5. Numerical method validation

The system of ODEs we need to solve can NOT be solved by hand. There is no known analytical solution. Before we solve it, let's make sure that our code works by solving a similar (but different) system of equations that DOES have a known analytical solution. (In a report, the code validation goes at the end in an Appendix, even though it is one of the first things that you will do when completing a project.)

To validate our numerical methods we hold the algal growth rate constant ($\mu = \mu_0$) such that Equation (2) is only a function of $A$ and not $P$: 

$$\frac{dA}{dt} = -K_{A1}A + \mu_0 A \tag{4}$$

Solving for $A$ using separation of variables gives

$$ A = A_0 e^{(\mu_0 - K_{A1})t} \tag{5}$$

Plugging Equation (5) into Equation (1) gives

$$\frac{dP}{dt} + K_{P1}P=  -K_{P2}\mu_0 A_0 e^{(\mu_0 - K_{A1})t} \tag{6}$$

which is a first order linear ODE that can be solved using the integrating factor $e^{K_{P1}t}$.

The solution for $P$ when algal growth rate is constant is 

$$ P(t) = P_0 e^{-K_{P1}t} - \frac{K_{P2} \mu_0 A_0}{\mu_0 - K_{A1} + K_{P1}} \left( e^{(\mu_0 - K_{A1})t} - e^{-K_{P1}t} \right) \tag{7}$$

### Question 5.1

Solve for $A(t)$ and $P(t)$ when the algal growth rate is constant ($\mu = \mu_0$) using RK4. Use the initial conditions and parameter values provided below.

I have provided some modifications to the RK4 code template which will allow your RK4 function to solve a system of ODEs! This means you can use the RK4 solver to solve for both $A$ and $P$ at each time step.

In [14]:
from matplotlib import pyplot as plt
import numpy as np

# parameters
Ka1 = 0.003 #1/days
Kp1 = 0.05 #1/days
Kp2 = 1. #1/days
mu0 = 0.43 #1/days

# initial conditions
A0 = 0.002 #mg/L
P0 = 0.9 #mg/L

# stream velocity
v_mps = 0.06 #m/s
v_mpd = v_mps*86400 #m/day

# max distance
xkm = 44.5 #km downstream
x = xkm*1000 #convert to m downstream

# step size in x
xstepkm = 0.5 #km
xstep = xstepkm*1000 #convert to m

# max time = max distance / stream velocity
tmax = (x/v_mpd)
#print(tmax)

# step size in t = step size in x / stream velocity
tstep = xstep/v_mpd #days
#print(tstep)

# number of steps = max time / step size
Nt = int(np.round(tmax/tstep)) 

# function for RHS of Phosphorus ODE (slope function)
def dPdt(P,A):
    return -Kp1*P - Kp2*mu0*A

# function for RHS of Algae ODE (slope function)
def dAdt(P,A):
    return -Ka1*A + mu0*A

# function to combine slope function evaluations
def der(t,y):
    P,A = y
    dP = dPdt(P,A)
    dA = dAdt(P,A)
    return np.array([dP, dA], dtype=float) 

# RK4 function
def solveODE_RK4(f,t0,y0,h,n_steps):
    # create time array
    t = t0 + h*np.arange(n_steps+1)
    # create storage array for the solutions
    y = np.full((n_steps+1, len(y0)), np.nan, dtype=float)
    # set the initial conditions for each solution
    y[0, :] = y0
    # After this you may copy your RK4 code from previous labs
    # YOUR CODE HERE
    
    return t,y #Note that y will contain solutions for both P and A

# solve the ODEs
time, PA = solveODE_RK4(der,0.0,(P0,A0),tstep,Nt)
P_RK4 = PA[:,0]
A_RK4 = PA[:,1]

### Question 5.2

Plot your RK4 solutions for $P$ and $A$ versus distance along the stream.

Also plot the analytical solutions for $P$ and $A$ versus distance along the stream.

HINT: If curves are overlapping, use dashed lines to distinguish them.

In [ ]:
# analytical solution for A
A_true = A0*np.exp((mu0 - Ka1)*time)
# analytical solution for P
P_true = P0*np.exp(-Kp1*time) - Kp2*mu0*A0*(np.exp((mu0 - Ka1)*time) - np.exp(-Kp1*time))/(mu0 - Ka1 + Kp1)
# convert from time to distance using velocity
x_dist = v_mpd*time #meters
x_dist_km = x_dist/1000 #kilometers

# Example
plt.plot(x_dist_km,P_true,'k-', label = 'P analytical')
# YOUR CODE HERE
# Include a legend and axis labels.
# Make sure your axis labels have units.
# Use LaTeX formatting
# Do not include a title

### Question 5.3

Describe in words how the phosphorus and algal concentrations evolve along the stream.

*Double click on this box and write your answer here*

## 6. Objective 1: Original alongstream phosphorus concentration

Once you are confident that you have your RK4 code working for systems of ODEs, go ahead and move on to solving the system of interest.

### Question 6.1

Solve the system when the growth rate is variable (Equations 1-3) using your RK4 method. 

Use the same parameters and initial conditions provided in the prior questions. All additional necessary parameters are provided below.

In [ ]:
mumax = 0.43 #1/days
Kp3 = 0.03 #1/days

# YOUR CODE HERE

### Question 6.2

Plot your RK4 solutions for $P$ and $A$ versus distance along the stream when the growth rate is variable (solutions to Equations 1-3).

Also plot the analytical solutions for $P$ and $A$ when the growth rate is constant (solutions to Equations 4 and 6).

In [ ]:
# YOUR CODE HERE
# Include a legend and axis labels.
# Make sure your axis labels have units.
# Use LaTeX formatting
# Do not include a title

### Question 6.3

How does using a variable growth rate influence the solution compared to using a constant growth rate for $\mu$?

*Double click on this box and write your answer here*

## 7. Objective 2: Original downstream phosphorus concentration

Remember: The requirement is that the phosphorus concentration in the stream must not exceed 0.05 $\frac{mg}{L}$ at a location 44.5 $km$ downstream of the discharge point.

Now we will investigate if the original effluent concentration of 0.90 $\frac{mg}{L}$ meets the new permit limit downstream.

### Question 7.1

Plot your RK4 solutions for $P$ versus distance along the stream when the growth rate is variable ($P$ solution to Equations 1-3).

Now add on a horizontal line at P=0.05 $\frac{mg}{L}$ representing the downstream limit.

In [ ]:
Plim = 0.05 #mg/L
# YOUR CODE HERE
# Include a legend and axis labels.
# Make sure your axis labels have units.
# Use LaTeX formatting
# Do not include a title

### Question 7.2

Store the value for the phosphorus concentration at the downstream location that you calculated using RK4 in the variable `Pdownstream`. (This is the solution that solved Equations 1-3.)

Print out the value.

In [ ]:
Pdownstream = #? YOUR CODE HERE

### Question 7.3

Does the original effluent concentration of 0.90 $\frac{mg}{L}$ meet the new downstream permit limit of 0.05 $\frac{mg}{L}$?

*Double click in this box and write your answer here*

## 8. Step size check

We need to make sure that our choice of step size is not significantly influencing our result. 

### Question 8.1

Recalculate your result for Question 7.2 using different time step sizes (provided below). 

Plot the time step size on the x axis and the result on the y axis.

In [ ]:
deltaN = 50
Nsteps = np.arange(Nt-deltaN, Nt+deltaN+1)
tsteps = tmax/Nsteps
Pdowns = np.full(len(tsteps),np.nan) #storage array for downstream phosphorus concentration
# YOUR CODE HERE
# Calcualte Pdowns using different time steps

# Plot Pdowns versus tsteps
plt.plot(tsteps,Pdowns,'*')
# Plot the original time step used too
plt.plot([tstep, tstep],[min(Pdowns),max(Pdowns)],'k-')
# Turn off scientific notation for easier reading
ax = plt.gca()
ax.ticklabel_format(style='plain', axis='y', useOffset=False)
# remove margins
plt.margins(0)
# Make sure your axis labels have units.
# Use LaTeX formatting
# Do not include a title
# YOUR CODE HERE (Make your plots look nice)

### Question 8.2

Make a similar plot, but this time plot percent change in the time step on the x axis,

and percent change in the result on the y axis.

Remember: 

% change $= \frac{new - original}{original}*100\%$

In [ ]:
# YOUR CODE HERE

plt.grid(True)
plt.xlabel('Change in time step $[\\%]$')
plt.ylabel('Change in downstream $P$ concentration $[\\%]$')

### Question 8.3

What do the plots you just made teach you about the accuracy of your method?

*Double click on this box and write your answer here*

## 9. Objective 3: Max allowable phosphorus effluent concentration

Now we need to find the maximum allowable phosphorus concentration that the treatment plant can discharge while still meeting the downstream limit.

### Question 9.1

Using the provided bisection root finding method, find the max allowable phosphorous effluent concentration ($P_{0max}$) by solving 

$$ P(x=44.5 km, P_0) = P_{lim}$$

where $P_{lim}$ is the downstream regulatory limit, and $P(x=44.5 km, P_0)$ means we are interested in the phosphorus concentration at the downstream location $x=44.5 km$, and how it is influenced by the effluent concentration $P_0$.

Another way to write it is

$$ P(t=t_{max}, P_0) - P_{lim} = 0$$

Print the result.


In [ ]:
def bisection(f, a, b, tol=1e-6, max_iter=100):
    """
    Find a root of f(x)=0 in the interval [a, b]
    using the bisection method.

    Requires 
    ----------
    f : function
        Function for which we want to find a root.
    a, b : float
        Interval endpoints. The function must change sign
        over [a, b], meaning f(a)*f(b) < 0.
    tol : float, optional
        Convergence tolerance. Iteration stops when either
        |f(c)| < tol or the interval width |b - a| < tol.
        Default is 10^(-6)
    max_iter : int, optional
        Maximum number of iterations allowed.
        Default is 100

    Returns
    -------
    root : float or None
        Approximate root location, or None if the method fails.
    """

    # evaluate the function at the edges of the interval
    fa = f(a)
    fb = f(b)

    root = None  # Default return value (in case no root is found)

    # Check that the interval actually brackets a root.
    # A root is guaranteed only if f(a) and f(b) have opposite signs.
    if fa * fb > 0:
        print("Error: f(a) and f(b) must have opposite signs.")
    else:
        # Perform up to max_iter iterations
        for i in range(max_iter):
            
            # Compute midpoint of current interval
            c = (a + b) / 2
            # Evaluate function at midpoint
            fc = f(c)

            # Check stopping criteria:
            # The function value is close enough to zero, or
            # the interval width is sufficiently small
            if abs(fc) < tol or abs(b - a) < tol:
                root = c
                break

            # Update the interval:
            # Determine which subinterval contains the root.
            # If f(a) and f(c) have opposite signs,
            # the root lies in [a, c]; otherwise in [c, b].
            if fa * fc < 0:
                b = c # Root is in left half
                fb = fc # Update function value at new endpoint
            else:
                a = c # Root is in right half
                fa = fc # Update function value at new endpoint

        # If no convergence occurred within max_iter
        if root is None:
            print("Warning: Maximum iterations reached.")
            root = c # Return best approximation obtained

    return root

def maxP(P0test): # We will test different values of P0
    time, PA = solveODE_RK4(der,0.0,(P0test,A0),tstep,Nt) # solve the ODE using the new P0
    P = PA[:,0] # grab the phosphorus solution
    Ptmax = P[-1] # grab the value of phosphorus conc at the last time (downstream regulation point)
    return Ptmax - Plim # We want to find the root of this equation

# YOUR CODE HERE
# P0max = ?

## 10. Objective 4: Max regulated alongstream phosphorus concentration

Now we need to look at the solution for when the effluent discharge meets the regulations.

### Question 10.1

Plot the solution for $P$ and $A$ (Equations 1-3) when the wastewater treatment plant discharges the max allowable phosphorus concentration $P_{0max}$. 

In [ ]:
# YOUR CODE HERE

### Question 10.2

Plot the solution for $P$ and $A$ (Equations 1-3) when the wastewater treatment plant discharges the max allowable phosphorus concentration $P_{0max}$. (Same as previous question.)

Additionally, plot the solution for $P$ and $A$ (Equations 1-3) using the original effluent concentration $P_{0}$

Also plot the downstream limit as a horizontal line on the plot. 

Use the same colors for the $P$ curves, and distinguish between the original and regulated $P$ curves using solid and dashed lines. Choose a differnet color for the $A$ curves, and distinguish between the original and regulated $A$ curves using solid and dashed lines. Plot the $P_{lim}$ as a thick solid line.

In [ ]:
# YOUR CODE HERE

### Question 10.3

Describe in words how the solution will change throughout the stream when the treatment plant changes their discharge practices to meet the new regulations.

*Double click on this box and write your answer here.*

## 11. Sensitivity to parameters

It is possible that some of our parameter values are not accurate. We need to test how sensitive our result is to changes in our parameters.

### Question 11.1

Recalculate your result for $P_{0max}$ using different values for stream velocity. 

Change stream veloicty by $\pm$ 20%.

Plot percent change in stream veloicty on the x-axis, 

and percent change in $P_{0max}$ on the y-axis.

Remember: 

% change $= \frac{new - original}{original}*100\%$

In [ ]:
pc = np.linspace(-20,20,100) #percent change
vs = v_mpd*(1+pc/100) # new velocity values
P0max_v = np.full(len(vs),np.nan) # storage array
for i in range(len(vs)): #loop over velocity values
    tstep = xstep/vs[i] #calculate new time step
    P0max_v[i] = bisection(maxP, 0, P0) #recalculate P0max
P0max_per_v = (P0max_v - P0max)*100/P0max #percent change in P0max
#return changed values to the original values for future
tstep = xstep/v_mpd

#YOUR CODE HERE
# Make sure your axis labels have units.
# Use LaTeX formatting
# Do not include a title

### Question 11.2

Recalculate your result for $P_{0max}$ using different values for initial algae concentration, $A_0$. 

Change algae concentration by $\pm$ 20%.

Plot percent change in algae concentration on the x-axis, 

and percent change in $P_{0max}$ on the y-axis.

In [ ]:
# YOUR CODE HERE

### Question 11.3

Recalculate your result for $P_{0max}$ using different values for the removal rate of phosphorus due to particle settling, $K_{P1}$. 

Change removal rate of phosphorus by $\pm$ 20%.

Plot percent change in removal rate of phosphorus on the x-axis, 

and percent change in $P_{0max}$ on the y-axis.

In [ ]:
# YOUR CODE HERE

### Question 11.4

Recalculate your result for $P_{0max}$ using different values for the yield coefficient, $K_{P2}$. 

Change removal of the yield coefficient by $\pm$ 20%.

Plot percent change in the yield coefficient on the x-axis, 

and percent change in $P_{0max}$ on the y-axis.

In [ ]:
# YOUR CODE HERE

### Question 11.5

Recalculate your result for $P_{0max}$ using different values for the maximum algal growth rate, $\mu_{max}$. 

Change removal of the max algal growth rate by $\pm$ 20%.

Plot percent change in the max algal growth rate on the x-axis, 

and percent change in $P_{0max}$ on the y-axis.

In [ ]:
# YOUR CODE HERE

### Question 11.6

Plot all of your sensitivity analysis curves on the same plot.

Plot percent change in parameter on the x-axis,

and percent change in $P_{0max}$ on the y-axis.

Use a legend to distinguish the curves.

In [ ]:
# YOUR CODE HERE

### Question 11.7

Which parameter is our result most sensitive to?

*Double click on this box and answer here.*

## 12. Conclusions

Using words, summarize the main conclusions reached by this lab.

Make sure to return to the list of objectives and clearly provide concluding statements for each objective. 

*Double click on this box and answer here.*

## 13. Recommendations

Using words, summarize the recommendations provided by this lab.

Recommendations should address:

* What to do with the main result. (In this case, What effluent limit $P_{0max}$ should the treatment facility adopt to meet the downstream water quality criterion? Briefly explain how you determined this value.)

* Limitations of the math model. Are there processes that the math model does not capture? (e.g. What if the velocity in the stream was not constant? Try to think of a few more examples yourself too.)

* Uncertainty in parameters. If the parameter values you used have uncertainty, how sensitive are your results to that uncertainty? (Hint: You did a sensitivity analysis to look at this.)

* Truncation error. All numerical methods have truncation error. How sensitive are your results to changes in step size? (Hint: You did a sensitivity analysis to look at this.)

* Round off error. Very small step sizes can increase round-off error. Based on your analysis, is round-off error significant enough to affect your conclusions? (Hint: You did a sensitivity analysis to look at this.)

*Double click on this box and answer here.*

## References

Chapra, S. C., & Canale, R. P. (2011). Numerical methods for engineers (Vol. 1221). New York: Mcgraw-hill.

Loucks, D. P. and Van Beek, E. (2017). Water resource systems planning and management:
An introduction to methods, models, and applications. Springer, Chapter 12: Water Quality Modelling and Prediction.

Nyholm, N. (1977). “Kinetics of phosphate limited algal growth.” Biotechnology and bioengineering,
19(4), 467–492.